<a href="https://colab.research.google.com/github/pytholabsbot1/cats_dogs_classifier/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gettin the Dataset

In [1]:
!pip install web-assist

In [2]:
import webAssist as wb

wb.Downloader('http://www.superdatascience.com/wp-content/uploads/2017/04/Convolutional_Neural_Networks.zip','cnn').download()

!unzip downlaoded_files/cnn.zip

appending to file -- cnn.zip
Archive:  downlaoded_files/cnn.zip
replace Convolutional_Neural_Networks/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-421ad32a3dda>", line 5, in <module>
    get_ipython().system('unzip downlaoded_files/cnn.zip')
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_shell.py", line 44, in system
    return _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=protected-access
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_system_commands.py", line 436, in _system_compat
    shell.var_expand(cmd, depth=2), clear_streamed_output=False)
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_system_commands.py", line 195, in _run_command
    return _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget)
  File "/usr/lib/python3.6/contextlib.py", line 88, in __exit__
    next(self.gen)
  File "/usr/local/lib/python3.6/dist-packages/googl

KeyboardInterrupt: ignored

## Dataset in Convolutional_Neural_Networks/dataset

In [0]:
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Convolutional_Neural_Networks/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('Convolutional_Neural_Networks/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000)

Using TensorFlow backend.


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
 282/8000 [>.............................] - ETA: 35:57 - loss: 0.6851 - acc: 0.5688

In [0]:
!ls Convolutional_Neural_Networks

cnn_homework_solution.py  cnn.py  dataset


##With TPU

In [0]:
import os

import tensorflow as tf
import numpy as np

In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

# Adding a second convolutional layer
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 50)        1400      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 50)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 31, 31, 50)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 50)        22550     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 50)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 50)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9800)              0         
__________

In [11]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

tpu_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Convolutional_Neural_Networks/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 500,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('Convolutional_Neural_Networks/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 500,
                                            class_mode = 'binary')

tpu_model.fit_generator(training_set,
                         steps_per_epoch = 40,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 4)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/10
40/40 [==============================] - 95s 2s/step - loss: 0.7379 - acc: 0.5040 - val_loss: 0.6913 - val_acc: 0.5862
Epoch 2/10
40/40 [==============================] - 86s 2s/step - loss: 0.6756 - acc: 0.5637 - val_loss: 0.6686 - val_acc: 0.5393
Epoch 3/10
40/40 [==============================] - 87s 2s/step - loss: 0.6412 - acc: 0.6257 - val_loss: 0.6223 - val_acc: 0.6648
Epoch 4/10
40/40 [==============================] - 87s 2s/step - loss: 0.6081 - acc: 0.6725 - val_loss: 0.5903 - val_acc: 0.7087
Epoch 5/10
40/40 [==============================] - 86s 2s/step - loss: 0.5832 - acc: 0.6917 - val_loss: 0.5609 - val_acc: 0.7218
Epoch 6/10
40/40 [==============================] - 88s 2s/step - loss: 0.5583 - acc: 0.7107 - val_loss: 0.5838 - val_acc: 0.7177
Epoch 7/10
40/40 [==============================] - 87s 2s/step - loss: 0.5384 - acc: 0.7297 - val_loss: 0.5644 - val_acc: 0.7283
Epoch 